In [17]:
import lightkurve as lk
import pandas as pd
import pathlib
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from ipywidgets import interact
import ipywidgets as widget

import statistics as st
import os.path
from os import path

from scipy import signal as sg
from scipy.ndimage.filters import generic_filter

#from google.colab import files
#%matplotlib qt5

print("Imports done.")

Imports done.


In [ ]:
import numpy as np
def outlier_removal(lc, sigma_value):
    lc = lc.remove_outliers(sigma = sigma_value)
    return lc #removes outliers

def plot_EPIC_ID(epic, bitmask, remove_outlier = True, sigma_value = 2.5):
    pixelfile = lk.search_targetpixelfile(epic).download(quality_bitmask=bitmask) #gets the data about the EPIC ID
    #fluxes = pixelfile.flux
    #print(np.std(fluxes[154]))
    #print(fluxes)
    
    #pixelfile.plot(frame = 128)
    #pixelfile.plot(frame = 0)
    fname = str(epic)+'.fits'
               
    lc = pixelfile.to_lightcurve(aperture_mask='all') #gets the light curve information
    if remove_outlier == True:
        lc = outlier_removal(lc, sigma_value) #calls the function that removes outliers
        print("Outliers removed")
    else:
        print("Outliers remain")
    lc.to_fits(path=fname, overwrite=True)
    #files.download(fname)    
    return lc, pixelfile #returns the light curve information

In [183]:
def epic_plotter(targets, bitmask_qual, remove_outlier):
    flux = []
    time = []
    flux_err = []
    filt = []
    
    # gets the light curve information for the ith EPIC ID in targets
    light_curve_info, pxfl = plot_EPIC_ID(targets, bitmask_qual, remove_outlier) 
    
    
    # gets the flux data about the ith EPIC ID, and adds it to the list of information 
    # about the fluxes of all the EPIC ID's
    flux.append(light_curve_info.flux)
    flux_err.append(light_curve_info.flux_err)
    time.append(light_curve_info.time) 
    filt = pd.notnull(flux)
    
   
   
    # creates a dataframe with the time and flux and flux_err
    data = pd.DataFrame({"Time":time[-1], 'Flux':flux[-1],'Flux_Err':flux_err[-1],'Filt':filt[-1]}) 
    flux_err = list(data['Flux_Err'])
    flux_err_mean = st.mean(flux_err)
    flux_err_limit = st.stdev(flux_err) * 3
    
    flux = list(data['Flux'])
    flux_mean = st.mean(flux)
    flux_sd = st.stdev(flux)
    flux_median = st.median(flux)
    flux_limit = flux_sd * 1
    print('flux mean: ', flux_mean, ' med :', flux_median, ' sd :',flux_sd)
   
    
    
    # sets the time as the index for finding the flux
    data.set_index(data.columns[0], inplace=True, drop = True) 
    
    filt2 = abs(data['Flux_Err'] -  flux_err_mean) < flux_err_limit
    filt3 = abs(data['Flux'] -  flux_mean) < flux_limit
    
    filt5 = data['Flux'] >  0
    filt6 = data['Flux'] < 20*flux_median
    
    data['Filt'] = data['Filt']  & filt2 & filt3 & filt4 & filt5 & filt6
    data.to_csv('out.csv',sep=',')
    
    data_fil = data[data['Filt']]
    print(data.shape)
    print(data_fil.shape)
    print('data fil:',data_fil.head())
    
    flux = list(data_fil['Flux'] )
    time = list(data_fil.index.values)
    
    total = len(flux)
    start = 0
    end = total
    sz=55
    #flux = generic_filter(flux, np.std, size=5)
    #flux = sg.medfilt(flux,kernel_size=sz)
    #flux = sg.wiener(flux,sz)
    
    data_fil['mod_flux'] = flux
    
    data_fil.to_csv('out2.csv',sep=',')
    
    sd = st.stdev(flux)
    mean = st.mean(flux)
    nSD=3
    limit = nSD*sd
    
  
    flux = flux[start:end]
    median = st.median(flux)
    for i in flux:
       i = i/median
    
    
    # gets the time information about the EPIC ID
    #flux = list(data[data['Filt']] )
    time = time[start:end]
    
    sz =[10,8]
    scale=1.3
    
    # plots using matplotlib
    #plt.gcf().set_size_inches(sz[0]*1.2,sz[1]/2.6)
    plt.plot(time, flux) 
    
    
    # light_curve_info.plot() #plots using the lightkurve.plot function
    pxfl.interact()
    
    plt.show() 
    
    #get cody graph from the paper
    img = '.\\..\\cody_graphs\\' +str(targets) + '.png'
    print(img)  
    
    if path.exists(img):
      plt.gcf().set_size_inches(sz[0]*scale,sz[1]*scale)
      plt.imshow(mpimg.imread(img))
    print('total points: ', total)
    print('data pts: ', len(flux), 'Start: ', start, 'End: ', end)
    print('Mean: ', st.mean(flux))
    print('Median: ', st.median(flux))
    print('SD: ', st.stdev(flux))

print("Functions defined.")

Functions defined.


In [184]:
base = 'https://raw.githubusercontent.com/SparshJohri/Kepler_Data_Analysis/master/'
file = 'Campaign_2_EPICS.csv'
campaign_epics = pd.read_csv(base+file)


file = "CodyTable.csv"
campaign_epics = list(campaign_epics[campaign_epics.columns[0]])
cody_epics = pd.read_csv(base+file)

cody_epics = [203954898]#cody_epics[cody_epics.columns[0]])


target_list = cody_epics
#target_list.append('203770673')

interact(epic_plotter, targets = target_list, bitmask_qual = ["default","hard", "none", "hardest"], \
         remove_outlier = [False, True])

interactive(children=(Dropdown(description='targets', options=(203954898,), value=203954898), Dropdown(descrip…

<function __main__.epic_plotter(targets, bitmask_qual, remove_outlier)>